# Object Detection(Car Detection)




### DOMAIN: 
Automotive , Surveillance
### CONTEXT: 
Computer vision can be used to automate supervision and generate action appropriate action trigger if the event is 
predicted from the image of interest. For example a car moving on the road can be easily identified by a camera as make of 
the car, type, colour, number plates etc
### DATA DESCRIPTION: 
The Cars dataset contains 16,185 images of 196 classes of cars. The data is split into 8,144 training images and 8,041 testing 
images, where each class has been split roughly in a 50-50 split. Classes are typically at the level of Make, Model, Year, e.g. 
2012 Tesla Model S or 2012 BMW M3 coupe
‣ Train Images: Consists of real images of cars as per the make and year of the car. 
‣ Test Images: Consists of real images of cars as per the make and year of the car.
‣ Train Annotation: Consists of bounding box region for training images.
‣ Test Annotation: Consists of bounding box region for testing images.
Dataset has been attached along with this project. Please use the same for this capstone project
Dataset: Original dataset link for your reference only: https://www.kaggle.com/jutrera/stanford-car-dataset-by-classes-folder
### REFERENCE: 
3D Object Representations for Fine-Grained Categorisation, Jonathan Krause, Michael Stark, Jia Deng, Li Fei-Fei 4th IEEE 
Workshop on 3D Representation and Recognition, at ICCV 2013 (3dRR-13). Sydney, Australia. Dec. 8, 2013.
### PROJECT OBJECTIVE: 
Design a DL based car identification model

In [ ]:
%tensorflow_version 2.x
#%tensorflow_version 1.x

In [ ]:
# Lets check version of tensor flow and keras installed 

import tensorflow
print(tensorflow.__version__)

2.8.0


In [ ]:
import keras
print(keras.__version__)

2.8.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


All the above versions libraries are required for the Third party implementation we are goin to use

## Legend

 ### Insights/inferences/results have been displayed post each step

In [ ]:
# Lets check if we are connected to GPU
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


## 1.0 Basic Config & Settings

### 1.1 Import necessary libraries

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import sys
import numpy as np
import seaborn as sns
import statistics as stats
sns.set(color_codes=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#Last import allows multiple outputs from one cell
import warnings
# Initialize the random number generator
import random
random.seed(0)

#Additional libs
import matplotlib.patches as patches


In [ ]:
# Useful Configuration/Setting

# suppress display of warnings
warnings.filterwarnings('ignore')

# display all dataframe columns
pd.options.display.max_columns = None

# to set the limit to 3 decimals
pd.options.display.float_format = '{:.7f}'.format

# display all dataframe rows
pd.options.display.max_rows = None

#Setting to shows all entries in array displayed
np.set_printoptions(threshold=sys.maxsize)

### 1.2 Set Necessary variables

In [ ]:
# Read The Data Set


images_path = "/content/drive/MyDrive/Capstone/Car Images/Car Images/"

#Lets note here that our images are structred as folders.Each folder is a cart type
#Corresponding folder will contain images of THAT car  type
#We have a seperate folder for Train and Test IMages

In [ ]:
# Variables required for model Building

images_height=300
images_width=300

## 2.0 Create Data Splits


In [ ]:
# Retrieve pickled data

X_Train_CompleteData = np.load("/content/drive/MyDrive/Capstone/Pickled_Info/X_Train.npy")
Y_Train_CompleteData  = np.load("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Train.npy")

In [ ]:
X_Train_CompleteData.shape
Y_Train_CompleteData.shape

(8144, 300, 300, 3)

(8144, 5)

In [ ]:
# Retrieve pickled data

#X_Test_CompleteData = np.load("/content/drive/MyDrive/Capstone/Pickled_Info/X_Test.npy")
#Y_Test_CompleteData = np.load("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Test.npy")

In [ ]:
# Let us work first with the train data 
# If we use both , we are seeing RAM crashes

In [ ]:
Y_Train_CompleteData_df=pd.DataFrame(data=Y_Train_CompleteData)

In [ ]:
Y_Train_CompleteData_df.head()

,0,1,2,3,4
0,37.0000000,56.0000000,276.0000000,268.0000000,184.0000000
1,18.0000000,21.0000000,277.0000000,258.0000000,184.0000000
2,18.0000000,135.0000000,184.0000000,225.0000000,184.0000000
3,61.0000000,72.0000000,196.0000000,245.0000000,184.0000000
4,3.0000000,5.0000000,283.0000000,263.0000000,184.0000000


In [ ]:
from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test=train_test_split(X_Train_CompleteData,Y_Train_CompleteData,test_size=0.2,random_state=1,stratify=Y_Train_CompleteData_df[4])
print("X_Train.shape",X_Train.shape)
print("X_Test.shape",X_Test.shape)
print("Y_Train.shape",Y_Train.shape)
print("Y_Test.shape",Y_Test.shape)

X_Train.shape (6515, 300, 300, 3)
X_Test.shape (1629, 300, 300, 3)
Y_Train.shape (6515, 5)
Y_Test.shape (1629, 5)


In [ ]:
#Y_Train_df=pd.DataFrame(data=Y_Train)

In [ ]:
'''from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test=train_test_split(X_Train,Y_Train,test_size=0.2,random_state=1,stratify=Y_Train_df[4])
print("X_Train.shape",X_Train.shape)
print("X_Test.shape",X_Test.shape)
print("Y_Train.shape",Y_Train.shape)
print("Y_Test.shape",Y_Test.shape)'''

'from sklearn.model_selection import train_test_split\nX_Train, X_Test, Y_Train, Y_Test=train_test_split(X_Train,Y_Train,test_size=0.2,random_state=1,stratify=Y_Train_df[4])\nprint("X_Train.shape",X_Train.shape)\nprint("X_Test.shape",X_Test.shape)\nprint("Y_Train.shape",Y_Train.shape)\nprint("Y_Test.shape",Y_Test.shape)'

In [ ]:
Y_Train[0:5]

array([[ 90.,  88., 284., 235., 140.],
       [ 14.,  40., 286., 280., 113.],
       [ 51.,  51., 255., 271.,  41.],
       [ 23.,  23., 282., 228.,   4.],
       [ 16.,  85., 285., 283.,  38.]])

In [ ]:

# NTS: to_pickle function not available , have to use save and in npy format 
np.save("/content/drive/MyDrive/Capstone/Pickled_Info/X_Train_Subset.npy",X_Train)
np.save("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Train_Subset.npy",Y_Train)

np.save("/content/drive/MyDrive/Capstone/Pickled_Info/X_Test_Subset.npy",X_Test)
np.save("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Test_Subset.npy",Y_Test)

## 3.0 Prepare Data

In [ ]:
X_Train=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/X_Train_Subset.npy")
Y_Train=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Train_Subset.npy")

X_Test=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/X_Test_Subset.npy")
Y_Test=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Test_Subset.npy")

In [ ]:
# Below we build our baseline n/w

In [ ]:
# First prepare data for Mobile Net model we will build

In [ ]:
X_Train.shape
Y_Train.shape
X_Test.shape
Y_Test.shape

(6515, 300, 300, 3)

(6515, 5)

(1629, 300, 300, 3)

(1629, 5)

In [ ]:
Y_Test[0:5]

array([[ 45.,  56., 252., 199.,  69.],
       [ 12., 111., 150., 196., 133.],
       [ 51.,  29., 282., 255.,  65.],
       [ 22.,  36., 289., 257., 160.],
       [ 40.,  19., 278., 279.,  58.]])

In [ ]:
# First lets convert it into integers all numbers 

Y_Train=Y_Train.astype(int)
Y_Test=Y_Test.astype(int)

In [ ]:
Y_Train[0:5]
Y_Test[0:5]

array([[ 90,  88, 284, 235, 140],
       [ 14,  40, 286, 280, 113],
       [ 51,  51, 255, 271,  41],
       [ 23,  23, 282, 228,   4],
       [ 16,  85, 285, 283,  38]])

array([[ 45,  56, 252, 199,  69],
       [ 12, 111, 150, 196, 133],
       [ 51,  29, 282, 255,  65],
       [ 22,  36, 289, 257, 160],
       [ 40,  19, 278, 279,  58]])

In [ ]:
# Prepare labels for Classification Head

In [ ]:
Y_Train_MobileNet_ClassInput=np.delete(Y_Train,obj=[0,1,2,3],axis=1)

In [ ]:
Y_Test_MobileNet_ClassInput=np.delete(Y_Test,obj=[0,1,2,3],axis=1)

In [ ]:
Y_Train_MobileNet_ClassInput.max()
print('\n')
Y_Train_MobileNet_ClassInput.min()
print('\n')
Y_Test_MobileNet_ClassInput.max()
print('\n')
Y_Test_MobileNet_ClassInput.min()
print('\n')

196

1

196

1

In [ ]:
# Classes dont start from 0 so
# First encode
#Then convert to OHV

In [ ]:
# Lets convert these labels to one hot vectors

# First encode the labels ,they are from 1 to 196, we want them to be from 0 to 195
# Deep learing : Lets try a simple ANN on the data 

from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y_Train_MobileNet_ClassInput)
Y_Train_MobileNet_ClassInput= encoder.transform(Y_Train_MobileNet_ClassInput)
Y_Test_MobileNet_ClassInput = encoder.transform(Y_Test_MobileNet_ClassInput)
#type(encoded_YTrain)

LabelEncoder()

In [ ]:
# Recheck max and min

Y_Train_MobileNet_ClassInput.max()
print('\n')
Y_Train_MobileNet_ClassInput.min()
print('\n')
Y_Test_MobileNet_ClassInput.max()
print('\n')
Y_Test_MobileNet_ClassInput.min()
print('\n')

# Looks good now  range(0,195)

195

0

195

0

In [ ]:
Y_Train_MobileNet_ClassInput[5]
print("\n")
Y_Train[5][4]

#Looks good

14

15

In [ ]:
# Now the same for Test

Y_Test_MobileNet_ClassInput[5]
print("\n")
Y_Test[5][4]

#Looks good

51

52

In [ ]:
## Label encoding has ben done correctly, lets proceed

In [ ]:
Y_Train_MobileNet_ClassInput.shape

(6515,)

In [ ]:
pd.DataFrame(Y_Train_MobileNet_ClassInput).value_counts().sort_index()

0      36
1      26
2      34
3      34
4      33
5      36
6      31
7      36
8      33
9      27
10     31
11     30
12     33
13     34
14     34
15     35
16     33
17     34
18     33
19     37
20     34
21     34
22     32
23     36
24     32
25     27
26     29
27     33
28     34
29     34
30     35
31     34
32     34
33     36
34     33
35     33
36     31
37     33
38     30
39     31
40     29
41     28
42     37
43     36
44     27
45     35
46     28
47     34
48     30
49     34
50     34
51     33
52     36
53     32
54     32
55     38
56     30
57     35
58     36
59     30
60     34
61     30
62     35
63     24
64     36
65     34
66     31
67     32
68     31
69     34
70     28
71     36
72     36
73     35
74     35
75     35
76     33
77     30
78     39
79     34
80     36
81     36
82     33
83     34
84     35
85     34
86     35
87     32
88     35
89     33
90     31
91     32
92     31
93     35
94     37
95     33
96     34
97     37
98     23
99     27


In [ ]:
pd.DataFrame(Y_Test_MobileNet_ClassInput).value_counts().sort_index()

0       9
1       6
2       9
3       8
4       8
5       9
6       8
7       9
8       8
9       6
10      7
11      7
12      8
13      9
14      9
15      9
16      8
17      9
18      8
19      9
20      8
21      9
22      8
23      9
24      8
25      7
26      7
27      8
28      9
29      8
30      9
31      8
32      8
33      9
34      8
35      8
36      8
37      8
38      7
39      8
40      7
41      7
42      9
43      9
44      6
45      9
46      7
47      9
48      8
49      8
50      9
51      8
52      9
53      8
54      8
55      9
56      8
57      9
58      9
59      7
60      9
61      7
62      9
63      6
64      9
65      8
66      8
67      8
68      7
69      9
70      7
71      9
72      9
73      9
74      9
75      9
76      8
77      8
78     10
79      9
80      9
81      9
82      8
83      8
84      9
85      9
86      9
87      8
88      9
89      8
90      8
91      8
92      8
93      9
94      9
95      8
96      8
97      9
98      5
99      7


In [ ]:
# Convert to OHV
# Convert above  to One hot vectors 
from tensorflow.keras.utils import to_categorical

Y_Train_MobileNet_ClassInput = to_categorical(Y_Train_MobileNet_ClassInput, num_classes=196)
Y_Test_MobileNet_ClassInput = to_categorical(Y_Test_MobileNet_ClassInput, num_classes=196)

print("Y_Train_MobileNet_ClassInput.shape",Y_Train_MobileNet_ClassInput.shape)
print("Y_Test_MobileNet_ClassInput.shape",Y_Test_MobileNet_ClassInput.shape)

Y_Train_MobileNet_ClassInput.shape (6515, 196)
Y_Test_MobileNet_ClassInput.shape (1629, 196)


In [ ]:
Y_Train_MobileNet_ClassInput[0]
Y_Test_MobileNet_ClassInput[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [ ]:
# Cross verify transofmration have been done correctly 
#On Train
Y_Train_MobileNet_ClassInput[0]
Originial=np.argmax(Y_Train_MobileNet_ClassInput[0])
print("\n")
print("Inverse Label",encoder.inverse_transform([Originial]))
print("Original",Y_Train[0][4])

#On Test
Y_Test_MobileNet_ClassInput[0]
Originial=np.argmax(Y_Test_MobileNet_ClassInput[0])
print("\n")
print("Inverse Label",(encoder.inverse_transform([Originial])))
print("Original",Y_Test[0][4])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)



Inverse Label [140]
Original 140


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)



Inverse Label [69]
Original 69


In [ ]:
### Now lets prepare labels for Regression Head

In [ ]:
# Lets remove labels of class
Y_Train_MobileNet_BoxInput=np.delete(Y_Train,obj=[4],axis=1)

In [ ]:
# Lets remove label of class
Y_Test_MobileNet_BoxInput=np.delete(Y_Test,obj=[4],axis=1)

In [ ]:
Y_Train_MobileNet_BoxInput.shape
Y_Test_MobileNet_BoxInput.shape

(6515, 4)

(1629, 4)

In [ ]:
'''# Lets prepare X data for Mobile net(will remain same for both reg and class head)
from PIL import Image
from keras.applications.mobilenet import preprocess_input

X_Train_MobileNet=X_Train.copy()
#for i, f in enumerate(X_Train):
  #X_Train_MobileNet[i] = preprocess_input(X_Train_MobileNet[i]) # Convert to float32 array'''

from keras.applications.mobilenet import preprocess_input
X_Train_MobileNet=preprocess_input(np.array(X_Train))
X_Test_MobileNet=preprocess_input(np.array(X_Test))


'# Lets prepare X data for Mobile net(will remain same for both reg and class head)\nfrom PIL import Image\nfrom keras.applications.mobilenet import preprocess_input\n\nX_Train_MobileNet=X_Train.copy()\n#for i, f in enumerate(X_Train):\n  #X_Train_MobileNet[i] = preprocess_input(X_Train_MobileNet[i]) # Convert to float32 array'

In [ ]:
'''X_Test_MobileNet=X_Test.copy()

#for i, f in enumerate(X_Test):
  #X_Test_MobileNet[i] = preprocess_input(np.array(X_Test_MobileNet[i], dtype=np.float32)) # Convert to float32 array'''

'X_Test_MobileNet=X_Test.copy()\n\n#for i, f in enumerate(X_Test):\n  #X_Test_MobileNet[i] = preprocess_input(np.array(X_Test_MobileNet[i], dtype=np.float32)) # Convert to float32 array'

In [ ]:
np.save("/content/drive/MyDrive/Capstone/Pickled_Info/X_Train_MobileNet.npy",X_Train_MobileNet)
np.save("/content/drive/MyDrive/Capstone/Pickled_Info/X_Test_MobileNet.npy",X_Test_MobileNet)

np.save("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Train_MobileNet_BoxInput.npy",Y_Train_MobileNet_BoxInput)

np.save("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Train_MobileNet_ClassInput.npy",Y_Train_MobileNet_ClassInput)

np.save("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Test_MobileNet_BoxInput.npy",Y_Test_MobileNet_BoxInput)

np.save("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Test_MobileNet_ClassInput.npy",Y_Test_MobileNet_ClassInput)


Seperate

In [ ]:
STOP

NameError: ignored

## 4.0 Retrive Pickled  Data Splits


In [ ]:
#X_Train=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/X_Train_Subset.npy")

X_Train_MobileNet=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/X_Train_MobileNet.npy")

X_Test_MobileNet=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/X_Test_MobileNet.npy")

Y_Train_MobileNet_BoxInput=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Train_MobileNet_BoxInput.npy")

Y_Train_MobileNet_ClassInput=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Train_MobileNet_ClassInput.npy")

Y_Test_MobileNet_BoxInput=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Test_MobileNet_BoxInput.npy")

Y_Test_MobileNet_ClassInput=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Test_MobileNet_ClassInput.npy")


In [ ]:
#X_Train.shape
X_Train_MobileNet.shape
X_Test_MobileNet.shape
Y_Train_MobileNet_BoxInput.shape
Y_Train_MobileNet_ClassInput.shape
Y_Test_MobileNet_BoxInput.shape
Y_Test_MobileNet_ClassInput.shape

(1628, 300, 300, 3)

(6516, 300, 300, 3)

(1628, 4)

(1628, 196)

(6516, 4)

(6516, 196)

In [ ]:
STOP

## 5.0 Model Building

In [ ]:
images_height=300
images_width=300

In [ ]:
# Lets build the model

from keras.applications.mobilenet import MobileNet
from keras.models import Model,load_model
from keras.layers import Conv2D, Reshape,Flatten,Dense,Dropout
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

ALPHA = 1.0

def create_model(trainable=True):
    regODModel = MobileNet(input_shape=(images_height, images_width, 3), include_top=False, alpha=ALPHA) # Load pre-trained mobilenet

    # to freeze layers, except the new top layer, of course, which will be added below
    for layer in regODModel.layers:
        layer.trainable = trainable
    
    # Add new top layer for Regression Head
    boxPredicModel = regODModel.layers[-1].output
    boxPredicModel = Flatten()(boxPredicModel)
    boxPredicModel = Dense(units=4)(boxPredicModel) # These are the 4 predicted coordinates of one BBox
    #boxPredicModel = Dense(units=4)(boxPredicModel) # These are the 4 predicted coordinates of one BBox

    ''' # Add new top layer for Classification  Head
    classPredicModel = model.layers[-1].output
    classPredicModel = Flatten()(classPredicModel)
    class_prediction = Dense(256, activation="relu")(classPredicModel)
    class_prediction = Dense(128, activation="relu")(class_prediction )
    class_prediction = Dropout(0.2)(class_prediction)
    class_prediction = Dense(64, activation="relu")(class_prediction)
    class_prediction = Dropout(0.2)(class_prediction )
    class_prediction = Dense(32, activation="relu")(class_prediction)
    classPredicModel = Dense(units=196,activation='softmax',name="class_output")(classPredicModel) # These are the 4 predicted coordinates of one BBox
    '''
    #Combine two heads 
    return Model(inputs=regODModel.input, outputs=boxPredicModel,name="box_output")
    #, name="boxoutput"

In [ ]:
# Define evaluation metric

def IOU(y_true, y_pred):
    intersections = 0
    unions = 0
    gt = y_true
    pred = y_pred
    # Compute interection of predicted (pred) and ground truth (gt) bounding boxes
    diff_width = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
    diff_height = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
    intersection = diff_width * diff_height

    # Compute union
    area_gt = gt[:,2] * gt[:,3]
    area_pred = pred[:,2] * pred[:,3]
    union = area_gt + area_pred - intersection
    # rk - again to repeat we know why he has minues , taken 2 times 

    # Compute intersection and union over multiple boxes
    for j, _ in enumerate(union):
      if union[j] > 0 and intersection[j] > 0 and union[j] >= intersection[j]:
        intersections += intersection[j]
        unions += union[j]

    # Compute IOU. Use epsilon to prevent division by zero
    iou = np.round(intersections / (unions + tensorflow.keras.backend.epsilon()), 4)# rounded to 4 
    # This must match the type used in py_func
    iou = iou.astype(np.float32)
    return iou
    # this is one number for the whole model

In [ ]:
def IoU(y_true, y_pred):
    iou = tensorflow.py_function(IOU, [y_true, y_pred], Tout=tensorflow.float32)
    return iou
# This is a wrapper funcction

In [ ]:
#Initialize the model and print summary

model = create_model(False) # Arg is False, if you want to freeze lower layers for fast training (but low accuracy)
model.summary() # Print summary

17235968/17225924 [==============================] - 0s 0us/step
Model: "box_output"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 150, 150, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 150, 150, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 150, 150, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 150, 150, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 150, 150, 32)     12

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=IoU)

In [ ]:
#Training the model
#Fit the model to the dataset

#Use early stopping,fit the model,give train data - training features and labels,batch size: 32 contd...
#...epochs: 10,give validation data - testing features and labels

# Use earlystopping
callback = EarlyStopping(monitor='val_IoU', patience=5, min_delta=0.001)
# If val_iou does not increase by 001 in 5 epochs , stop
# Fit the model
model.fit(X_Train_MobileNet, Y_Train_MobileNet_BoxInput, validation_data=(X_Test_MobileNet, Y_Test_MobileNet_BoxInput), epochs=5, batch_size=700)

In [ ]:
model.load_weights('/content/drive/MyDrive/Capstone/Pickled_Info/RegODModel_bestWieghts_300.h5')

In [ ]:
#model.save('/content/drive/MyDrive/Capstone/Pickled_Info/RegODModel_300.h5')
#model.save_weights('/content/drive/MyDrive/Capstone/Pickled_Info/RegODModel_bestWieghts_300.h5')

In [ ]:
#Final loss and accuracy
model.evaluate(X_Test_MobileNet, Y_Test_MobileNet_BoxInput)

204/204 [==============================] - 31s 81ms/step - loss: 3423.6929 - IoU: 0.4701


[3423.69287109375, 0.4701077938079834]

In [ ]:
# Take some sample image and predict

region=model.predict(X_Test_MobileNet[200].reshape(1,300,300,3)) # Predict the BBox

In [ ]:
region

In [ ]:
# Lets visually display any two train data samples

# View any one sample of prepared Train Data 
im=X_Test_MobileNet[200]
# Show resized image with Box
fig,ax = plt.subplots(1)
ax.imshow(im/255)
res_x0=region[0][0]
res_y0=region[0][1]
res_x1=region[0][2]
res_y1=region[0][3]
rect = patches.Rectangle((res_x0, res_y0), res_x1 - res_x0, res_y1 - res_y0, linewidth=2, edgecolor='r', facecolor='none')
ax.add_patch(rect)
plt.show()
